# Test Env

Import data.

In [12]:
%load_ext autoreload
%autoreload 2

import dgym as dg

# load all data
print('load data')
path = '../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

reactions = dg.ReactionCollection.from_json(
    path = f'{path}/All_Rxns_rxn_library.json',
    smarts_col = 'reaction_string',
    classes_col = 'functional_groups'
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
load data


In [38]:
from dgym.envs.oracle import DGLOracle

lipo_oracle = DGLOracle('GCN_canonical_Lipophilicity')
esol_oracle = DGLOracle('GCN_canonical_ESOL')

In [40]:
lipo_oracle.predict(deck)

[1.7650638818740845,
 0.7854697108268738,
 1.7408784627914429,
 1.9714215993881226,
 1.7279001474380493,
 1.8764628171920776,
 -0.21282707154750824,
 1.238732933998108,
 2.3975718021392822,
 1.8674098253250122,
 1.922244668006897,
 -0.3436444401741028,
 0.5199243426322937,
 0.49248164892196655,
 2.2068402767181396,
 1.063495397567749,
 1.5084296464920044,
 1.4621894359588623,
 1.192696213722229,
 2.067012310028076,
 2.4371676445007324,
 2.3089582920074463,
 1.7162035703659058,
 0.8927620649337769,
 0.4615725874900818,
 1.2812491655349731,
 2.943678140640259,
 -0.0202132910490036,
 0.6155068278312683,
 2.029090404510498,
 0.34735870361328125,
 2.7589919567108154,
 1.5219932794570923,
 1.2238348722457886,
 1.0271803140640259,
 2.017866611480713,
 0.22862263023853302,
 1.5600193738937378,
 -0.0238104909658432,
 -0.1671995371580124,
 0.7439402937889099,
 2.1178598403930664,
 0.9134944677352905,
 2.1844522953033447,
 2.3933446407318115,
 1.8493019342422485,
 -0.3441724181175232,
 0.90734702

In [2]:
building_blocks = dg.datasets.enamine(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630.sdf')
fingerprints = dg.datasets.fingerprints(f'{path}/out/Enamine_Building_Blocks_Stock_262336cmpd_20230630.fpb')

# align fingerprints to building blocks
print('align fingerprints')
fingerprints = dg.utils.sort_fingerprints(fingerprints, building_blocks)

# partition building blocks according to functional groups
print('partition building blocks')
templates = dg.utils.get_unique_reactants(reactions)
building_blocks = dg.utils.partition_building_blocks(building_blocks, templates, out_dir=path)

align fingerprints
partition building blocks


In [3]:
from dgym.envs.library_designer import LibraryDesigner

library_designer = LibraryDesigner(
    reactions,
    building_blocks,
    fingerprints
)

In [4]:
from dgym.envs import DrugEnv

drug_env = DrugEnv(library_designer, library=list(deck[:6]))
# is there data associated with the library? could be.

action = {
    'design': {
        'selected_molecules': [0],
        'num_analogs': 2,
        'percent_random': 0.0
        # fraction_random
    }
}

drug_env.step(action)
# [display(p) for p in drug_env.library[6:]]

([<dgym.molecule.Molecule at 0x7f34020629d0>,
 None,
 None,
 {})

In [32]:
from dgym.agents import DrugAgent

In [33]:
drug_agent = DrugAgent(drug_env.action_space)

In [54]:
dg.envs.

NameError: name 'dgym' is not defined

In [179]:
drug_agent.act()

TypeError: DrugAgent.act() missing 2 required positional arguments: 'observations' and 'mask'